# Notebook: Convert Model Results to Latex

This notebook is used to load the .json files with the model performance in order to convert them into a latex table for the paper

In [410]:
# Todo: Schauen, ob es die Metriken auch bei anderen Modellen gibt
# Todo: 1.000 <- Punkte einfügen
# Todo: Soll bei nur Real gehen
# Todo: Soll bei allen Tasks gehen
# Todo: bei f1 micro etc 3 nachkommastellen
# Schauen, dass es bei jedem Task "eval_f1_micro", "eval_f1_macro", "accuracy"

## Packages

In [411]:
import json
import numpy as np

## Settings / Constants

In [412]:
RESULTS_PATH_BASE = "../07 train models/results_json/results_"
LLMS = ["Llama70B", "GPT-3"]
# , "aspect_category_sentiment", "end_2_end_absa" ,"target_aspect_sentiment_detection"]
ABSA_TASKS = ["aspect_category", "aspect_category_sentiment", "end_2_end_absa", "target_aspect_sentiment_detection"]
SYNTH_COMBINATIONS = {
    "random": [
        {"real": 500, "synth": 500},
        {"real": 500, "synth": 1000},
        {"real": 500, "synth": 1500}
    ], "fixed": [
        {"real": 25, "synth": 475},
        {"real": 25, "synth": 975},
        {"real": 25, "synth": 1975}
    ]}

In [413]:
LLMS_ENCODED = {"GPT-3": "\\textbf{GPT-3.5-turbo}", "Llama70B": "\\textbf{Llama-2-70B}"}
ENCODE_CONDITION = {"fixed": "\\textbf{LRS\\textsubscript{25}}",
                    "random": "\\textbf{LRS\\textsubscript{500}}"}

N_METRICS = {"aspect_category": 6, "aspect_category_sentiment": 6,
             "end_2_end_absa": 6, "target_aspect_sentiment_detection": 5}

## Code

In [414]:
def round_number(num, decimal_places):
    formatted_num = "{:.{}f}".format(num, decimal_places)
    rounded_num_str = "{:.{}f}".format(float(formatted_num), decimal_places)
    return rounded_num_str

def add_thousand_dots(n_sample):
    if isinstance(n_sample, str):
        if '.' in n_sample:
            integer_part, decimal_part = n_sample.split('.')
            formatted_integer_part = "{:,}".format(int(integer_part))
            result = f"{formatted_integer_part}.{decimal_part}"
        else:
            result = "{:,}".format(int(n_sample))
    elif isinstance(n_sample, np.float64):
        result = "{:,}".format(round(n_sample, 1))
    else:
        result = n_sample
    
    return result

### Report Main Metrics

In [415]:
def get_metric(metric_name, results):
    main_metric = add_thousand_dots(round_number(results[metric_name], 3))
    std_metric = round_number(np.std([res[metric_name]* 100 for res in results["single_split_results"]]), 2)
    return main_metric + "\\textsubscript{" + std_metric + "}"

In [416]:
get_metric("eval_f1_micro", results)

'0.717\\textsubscript{3.59}'

In [417]:
for absa_task in ABSA_TASKS:
    print(absa_task, ":\n\n", "-------#-----#-----#-------")
    for n_real_idx, n_real in enumerate([500, 1000, 2000]):
        if n_real_idx == 0:
            condition_print = "\\multirow{3}{*}{\\textbf{Real Examples}} & \\multirow{3}{*}{-}"
        else:
            condition_print = " & "

        json_path = RESULTS_PATH_BASE + \
            f"only_real_real{n_real}_synth0_{absa_task}_random.json"

        with open(json_path, 'r') as json_file:
            results = json.load(json_file)

        if absa_task != "target_aspect_sentiment_detection":
            print(
                f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 & {get_metric('eval_f1_micro', results)} & {get_metric('eval_f1_macro', results)} & {get_metric('eval_accuracy', results)} \\\\")
        else:
            print(
                f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 & {get_metric('eval_f1', results)} & {get_metric('eval_accuracy', results)} \\\\")
    print("\\hline")
    for llm_idx, llm in enumerate(LLMS):
        for fs_idx, few_shot_condition in enumerate(["fixed", "random"]):
            for freq_idx, freq in enumerate(SYNTH_COMBINATIONS[few_shot_condition]):
                n_real = freq["real"]
                n_synth = freq["synth"]
                json_path = RESULTS_PATH_BASE + llm + \
                    f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
                with open(json_path, 'r') as json_file:
                    results = json.load(json_file)

                if fs_idx == 0 and freq_idx == 0:
                    llm_print = "\\multirow{6}{*}{"+LLMS_ENCODED[llm]+"}"
                else:
                    llm_print = ""

                if freq_idx == 0:
                    condition_print = "\\multirow{3}{*}{" + \
                        ENCODE_CONDITION[few_shot_condition]+"}"
                else:
                    condition_print = ""

                if absa_task != "target_aspect_sentiment_detection":
                    f1_metrics = f"{get_metric('eval_f1_micro', results)} & {get_metric('eval_f1_macro', results)}"
                else:
                    f1_metrics = f"{get_metric('eval_f1', results)}"
                print(
                    f"{llm_print} & {condition_print} & {add_thousand_dots(str(n_real))} & {add_thousand_dots(str(n_synth))} & {f1_metrics} & {get_metric('eval_accuracy', results)} \\\\")

                # print(llm_idx, fs_idx, freq_idx)
            if freq_idx == 2:
                print(
                    "\\arrayrulecolor{gray}\cline{2-"+str(N_METRICS[absa_task]+1)+"}\\arrayrulecolor{black}")
            else:
                print("\\hline")
        print("\hline")
    print("\n\n")

aspect_category :

 -------#-----#-----#-------
\multirow{3}{*}{\textbf{Real Examples}} & \multirow{3}{*}{-} & 500 & 0 & 0.909\textsubscript{1.37} & 0.900\textsubscript{1.66} & 0.838\textsubscript{2.01} \\
 &  & 1,000 & 0 & 0.920\textsubscript{1.19} & 0.911\textsubscript{1.46} & 0.861\textsubscript{1.63} \\
 &  & 2,000 & 0 & 0.924\textsubscript{1.15} & 0.915\textsubscript{1.31} & 0.862\textsubscript{1.86} \\
\hline
\multirow{6}{*}{\textbf{Llama-2-70B}} & \multirow{3}{*}{\textbf{LRS\textsubscript{25}}} & 25 & 475 & 0.776\textsubscript{2.16} & 0.768\textsubscript{2.53} & 0.626\textsubscript{3.27} \\
 &  & 25 & 975 & 0.778\textsubscript{2.18} & 0.762\textsubscript{2.13} & 0.638\textsubscript{3.92} \\
 &  & 25 & 1,975 & 0.784\textsubscript{1.99} & 0.769\textsubscript{1.55} & 0.664\textsubscript{2.24} \\
\arrayrulecolor{gray}\cline{2-7}\arrayrulecolor{black}
 & \multirow{3}{*}{\textbf{LRS\textsubscript{500}}} & 500 & 500 & 0.886\textsubscript{1.54} & 0.874\textsubscript{1.44} & 0.798\textsu

### Report Metrics Fine-Grained Metrics

#### ACD

In [418]:
absa_task = "aspect_category"
for n_real_idx, n_real in enumerate([500, 1000, 2000]):
    json_path = RESULTS_PATH_BASE + \
        f"only_real_real{n_real}_synth0_{absa_task}_random.json"
    with open(json_path, 'r') as json_file:
        results = json.load(json_file)

    if n_real_idx == 0:
        condition_print = "\\multirow{3}{*}{\\textbf{Real Examples}} & \\multirow{3}{*}{-}"
    else:
        condition_print = " & "

    class_wise_metrics = ""
    for ac in ["GENERAL-IMPRESSION", "FOOD", "SERVICE", "AMBIENCE", "PRICE"]:
        for metric in ["f1", "accuracy"]:
            class_wise_metrics += f"{get_metric(f'eval_{metric}_{ac}', results)} &"
    print(
        f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 & {class_wise_metrics[:-1]} \\\\")
print("\\hline")
for fs_idx, few_shot_condition in enumerate(SYNTH_COMBINATIONS.keys()):
    for llm_idx, llm in enumerate(LLMS):
        for freq_idx, freq in enumerate(SYNTH_COMBINATIONS[few_shot_condition]):
            n_real = freq["real"]
            n_synth = freq["synth"]
            json_path = RESULTS_PATH_BASE + llm + \
                f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
            with open(json_path, 'r') as json_file:
                results = json.load(json_file)

            if llm_idx == 0 and freq_idx == 0:
                llm_print = "\\multirow{6}{*}{"+LLMS_ENCODED[llm]+"}"
            else:
                llm_print = ""

            if freq_idx == 0:
                condition_print = "\\multirow{3}{*}{" + \
                    ENCODE_CONDITION[few_shot_condition]+"}"
            else:
                condition_print = ""

            class_wise_metrics = ""
            for ac in ["GENERAL-IMPRESSION", "FOOD", "SERVICE", "AMBIENCE", "PRICE"]:
                for metric in ["f1", "accuracy"]:
                    class_wise_metrics += f"{get_metric(f'eval_{metric}_{ac}', results)} &"

            print(
                f"{llm_print} & {condition_print} & {add_thousand_dots(str(n_real))} & {add_thousand_dots(str(n_synth))} & {class_wise_metrics[:-1]} \\\\")
            if freq_idx == 2 and llm_idx == 0:
               print(
                "\\arrayrulecolor{gray}\cline{2-14}\\arrayrulecolor{black}")

    print("\hline")

\multirow{3}{*}{\textbf{Real Examples}} & \multirow{3}{*}{-} & 500 & 0 & 0.852\textsubscript{3.71} &0.930\textsubscript{1.60} &0.936\textsubscript{0.90} &0.945\textsubscript{0.78} &0.930\textsubscript{1.84} &0.954\textsubscript{1.35} &0.873\textsubscript{3.80} &0.964\textsubscript{1.16} &0.908\textsubscript{3.57} &0.985\textsubscript{0.60}  \\
 &  & 1,000 & 0 & 0.867\textsubscript{2.65} &0.935\textsubscript{1.30} &0.947\textsubscript{0.92} &0.955\textsubscript{0.85} &0.941\textsubscript{1.33} &0.962\textsubscript{0.93} &0.882\textsubscript{2.87} &0.967\textsubscript{0.89} &0.917\textsubscript{2.42} &0.987\textsubscript{0.47}  \\
 &  & 2,000 & 0 & 0.874\textsubscript{2.77} &0.938\textsubscript{1.18} &0.949\textsubscript{0.82} &0.957\textsubscript{0.76} &0.944\textsubscript{1.10} &0.964\textsubscript{0.79} &0.883\textsubscript{2.96} &0.967\textsubscript{0.98} &0.926\textsubscript{3.00} &0.988\textsubscript{0.53}  \\
\hline
\multirow{6}{*}{\textbf{Llama-2-70B}} & \multirow{3}{*}{\textbf{L

#### ACSA

In [419]:
absa_task = "aspect_category_sentiment"

idx = 0
for ac_idx, aspect_categories in enumerate([["GENERAL-IMPRESSION", "FOOD"], ["SERVICE", "AMBIENCE"], ["PRICE"]]):
    print("Table for:", aspect_categories, "\n\n")
    for n_real_idx, n_real in enumerate([500, 1000, 2000]):
        json_path = RESULTS_PATH_BASE + \
            f"only_real_real{n_real}_synth0_{absa_task}_random.json"
        with open(json_path, 'r') as json_file:
            results = json.load(json_file)

        if n_real_idx == 0:
            condition_print = "\\multirow{3}{*}{\\textbf{Real Examples}} & \\multirow{3}{*}{-}"
        else:
            condition_print = " & "

        condition_string = f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 &"
        metrics_class_wise = ""
        for ac in aspect_categories:
            for polarity in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                for metric in ["eval_f1", "eval_accuracy"]:
                    metrics_class_wise += f" {get_metric(f'{metric}_{ac}-{polarity}', results)} &"

        print(condition_string + metrics_class_wise[:-1] + "\\\\")

    print("\\hline")
    for fs_idx, few_shot_condition in enumerate(SYNTH_COMBINATIONS.keys()):
        for llm_idx, llm in enumerate(LLMS):
            for freq_idx, freq in enumerate(SYNTH_COMBINATIONS[few_shot_condition]):
                n_real = freq["real"]
                n_synth = freq["synth"]
                json_path = RESULTS_PATH_BASE + llm + \
                    f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
                with open(json_path, 'r') as json_file:
                    results = json.load(json_file)

                if llm_idx == 0 and freq_idx == 0:
                    llm_print = "\\multirow{6}{*}{"+LLMS_ENCODED[llm]+"}"
                else:
                    llm_print = ""

                if freq_idx == 0:
                    condition_print = "\\multirow{3}{*}{" + \
                        ENCODE_CONDITION[few_shot_condition]+"}"
                else:
                    condition_print = ""

                condition_string = f"{llm_print} & {condition_print} & {add_thousand_dots(str(n_real))} & {add_thousand_dots(str(n_synth))} &"
                metrics_class_wise = ""
                for ac in aspect_categories:
                    for polarity in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                        for metric in ["eval_f1", "eval_accuracy"]:
                            metrics_class_wise += f" {get_metric(f'{metric}_{ac}-{polarity}', results)} &"

                print(condition_string + metrics_class_wise[:-1] + "\\\\")
            
                n_col = 10 if ac_idx == 2 else 16
                if freq_idx == 2:
                   print("\\arrayrulecolor{gray}\cline{2-"+str(n_col)+"}\\arrayrulecolor{black}")
        print("\hline")
    idx += 1

Table for: ['GENERAL-IMPRESSION', 'FOOD'] 


\multirow{3}{*}{\textbf{Real Examples}} & \multirow{3}{*}{-} & 500 & 0 & 0.829\textsubscript{3.05} & 0.958\textsubscript{0.41} & 0.000\textsubscript{0.00} & 0.992\textsubscript{0.33} & 0.822\textsubscript{7.34} & 0.963\textsubscript{1.48} & 0.919\textsubscript{1.48} & 0.961\textsubscript{0.84} & 0.663\textsubscript{8.56} & 0.981\textsubscript{0.49} & 0.822\textsubscript{4.06} & 0.944\textsubscript{1.19} \\
 &  & 1,000 & 0 & 0.847\textsubscript{2.43} & 0.961\textsubscript{0.56} & 0.571\textsubscript{26.45} & 0.994\textsubscript{0.45} & 0.852\textsubscript{6.61} & 0.969\textsubscript{1.30} & 0.945\textsubscript{1.10} & 0.973\textsubscript{0.57} & 0.816\textsubscript{5.95} & 0.986\textsubscript{0.41} & 0.869\textsubscript{2.86} & 0.957\textsubscript{0.93} \\
 &  & 2,000 & 0 & 0.856\textsubscript{3.62} & 0.963\textsubscript{0.81} & 0.677\textsubscript{20.57} & 0.995\textsubscript{0.40} & 0.865\textsubscript{4.17} & 0.971\textsubscript{0.96} & 0.